In [ ]:
import requests
import json
import csv
import pandas as pd
from pandas.io.json import json_normalize


z = pd.read_csv(
    "C:\\Users\\muhammad.madyun\\OneDrive - Hayes Locums\\Documents\\Data\\Analysis\\Sourcing\\AAOS.csv",
    low_memory=False,
    index_col=False,
    encoding="unicode_escape",
).fillna("")
s = pd.read_csv(
    "C:\\Users\\muhammad.madyun\\OneDrive - Hayes Locums\\Documents\\Data\\Analysis\\ama.csv",
    low_memory=False,
    index_col=False,
    encoding="unicode_escape",
).fillna("")
# acct = pd.read_csv(
#    "C:\\Users\\muhammad.madyun\\OneDrive - Hayes Locums\\Documents\\Data\\Analysis\\acct_npi.csv",
#    low_memory=False,
#    index_col=False,
#    encoding="unicode_escape",
# ).fillna("")
cross = pd.read_csv(
    "C:\\Users\\muhammad.madyun\\OneDrive - Hayes Locums\\Documents\\Data\\Analysis\\cross_npi.csv",
    low_memory=False,
    index_col=False,
    encoding="unicode_escape",
    dtype="object",
).fillna("")


cross = cross[cross["NPI"] != ""]
# acct1 = acct[acct["Person Account: NPI Number"] != ""]
s.rename(columns={"AMA Specialty: AMA Specialty Name": "Specialization"})
s = s.drop_duplicates()
s1 = s[s["Taxonomy Code"] != ""]


errors = []
data = []

# merge = pd.concat([a,b,c], ignore_index=True)
# file1 = merge[(merge['Mail Name'] != '© 2023 by the American Medical Association') & merge['Mail Name'] != 'Â© 2023 by the American Medical Association']
# file1 = file1[['Mail  Last Name', 'Mail First Name', 'MD-DO Flag', 'City', 'State', 'Zip', 'Primary Specialty']]
# file = file1.rename(
#    columns = {'Mail  Last Name': 'last_name',
#            'Mail First Name': 'first_name',
#            'Mail City': 'City',
#            'State': 'state',
#            'Zip': 'Postal Code',
#            'MD-DO Flag': 'Practitioner Type'
#            })
data_dfs = pd.DataFrame(
    columns=[
        "number",
        "First Name",
        "Last Name",
        "Gender",
        "basic.enumeration_date",
        "country_name",
        "address_purpose",
        "address_1",
        "city",
        "state_x",
        "postal_code",
        "telephone_number",
        "code",
        "taxonomy_group",
        "desc",
        "state_y",
        "license",
        "primary",
    ]
).fillna("")

data_df = pd.DataFrame(
    columns=[
        "number",
        "First Name",
        "Last Name",
        "Gender",
        "basic.enumeration_date",
        "country_name",
        "address_purpose",
        "address_1",
        "city",
        "state_x",
        "postal_code",
        "telephone_number",
        "code",
        "taxonomy_group",
        "desc",
        "state_y",
        "license",
        "primary",
    ]
).fillna("")

url = "https://npiregistry.cms.hhs.gov/api/?version=2.1"


file = z.fillna("")

file_df = file[["First Name", "Last Name"]]

for row in file_df.index:
    first_name, last_name = (
        file_df["First Name"][row],
        file_df["Last Name"][row],
    )
    # file_df["Last Name"].str[:3][row]
    parameters = {
        "version": "2.1",
        "first_name": first_name,
        "last_name": last_name,
        "pretty": "true",
    }
    # raises exception when not a 2xx response

    response = requests.get(url, params=parameters).json()
    # response.raise_for_status()
    #
    # if response.status_code != 204:
    #    return response.json()
    for i in response:
        if i == "result_count":
            if response["result_count"] < 1:
                errors.append(parameters)
            elif response["result_count"] >= 1:
                data.append(response)
        else:
            errors.append(parameters)
    for l in data:
        data1 = pd.json_normalize(
            l["results"], "addresses", ["number", "enumeration_type"]
        )
        data1 = data1.drop_duplicates(subset="number")
        data2 = pd.json_normalize(l["results"], "taxonomies", ["number"])
        data4 = pd.json_normalize(l["results"])
        file1 = pd.merge_ordered(data1, data2, on="number", how="outer")
        file2 = pd.merge_ordered(file1, data4, on="number", how="outer")

        file3 = file2[
            [
                "number",
                "basic.first_name",
                "basic.last_name",
                "basic.gender",
                "basic.enumeration_date",
                "country_name",
                "address_purpose",
                "address_1",
                "city",
                "state_x",
                "postal_code",
                "telephone_number",
                "code",
                "taxonomy_group",
                "desc",
                "state_y",
                "license",
                "primary",
            ]
        ]

        file4a = file3.rename(
            columns={
                "basic.first_name": "First Name",
                "basic.last_name": "Last Name",
                "basic.middle_name": "Middle Name",
                "basic.credential": "Practitioner Type",
                "basic.gender": "Gender",
            }
        ).drop_duplicates(subset="number")
        file4b = pd.merge_ordered(
            file4a, cross, left_on="number", right_on="NPI", how="left"
        ).fillna("")
        file4c = pd.merge_ordered(
            file4b, s1, left_on="code", right_on="Taxonomy Code", how="left"
        ).fillna("")
        file4c = file4c.replace(
            {"Practitioner Type": ""}, {"Practitioner Type": "MD"}, regex=True
        )
        file4 = file4c[
            file4c["AMA Specialty: AMA Specialty Name"] != ""
        ].drop_duplicates(subset="number")

        if len(data_df) < 500:
            data_df = pd.concat([data_df, file4]).drop_duplicates(subset="number")

        else:
            data_df.to_csv(" e1.csv", index=False)
            break

In [77]:
data_df

,number,First Name,Last Name,Gender,basic.enumeration_date,country_name,address_purpose,address_1,city,state_x,...,Practitioner Type,Active IMLC,Licenses (Active),Sub Specialty,AMA Specialty: AMA Specialty Name,AMA Specialty: ID,NS Internal Id,Classification,GL Code,Taxonomy Code
0,1396743779,DAVID,ADKISON,NaN,2005-07-11,United States,LOCATION,2700 10TH AVE S,BIRMINGHAM,AL,...,MD,,AL; CA,General Orthopedic Surgery,Orthopaedic Surgery,a3k7h000000NMX5,106.0,Orthopaedic Surgery,02-1 Orthopedic Surgery,207X00000X
0,1134127046,CARL,ALBERTSON,NaN,2005-07-13,United States,MAILING,PO BOX 1186,FAIRHOPE,AL,...,MD,,AL,General Orthopedic Surgery,Orthopaedic Surgery,a3k7h000000NMX5,106.0,Orthopaedic Surgery,02-1 Orthopedic Surgery,207X00000X
1,1376598458,HOWARD,ALEXANDER,NaN,2006-05-24,United States,LOCATION,4715 WHITESBURG DR S,HUNTSVILLE,AL,...,MD,,,General Orthopedic Surgery,Orthopaedic Surgery,a3k7h000000NMX5,106.0,Orthopaedic Surgery,02-1 Orthopedic Surgery,207X00000X
1,1982605366,DAVID,ALFORD,NaN,2005-08-09,United States,MAILING,PO BOX 729,DOTHAN,AL,...,MD,,,General Orthopedic Surgery,Orthopaedic Surgery,a3k7h000000NMX5,106.0,Orthopaedic Surgery,02-1 Orthopedic Surgery,207X00000X
0,1508891516,LANDON,ANDERSON,NaN,2006-07-11,United States,MAILING,PO BOX 218,ANDALUSIA,AL,...,MD,,,General Orthopedic Surgery,Orthopaedic Surgery,a3k7h000000NMX5,106.0,Orthopaedic Surgery,02-1 Orthopedic Surgery,207X00000X
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4,1831326479,ASHISH,SHAH,NaN,2009-06-21,United States,LOCATION,24411 HEALTH CENTER DR,LAGUNA HILLS,CA,...,DO,,MA; RI,Clinical Cardiac Electrophysiology,Clinical Cardiac Electrophysiology,a3k7h000000NMYa,140.0,Clinical Cardiac Electrophysiology,10-2 Cardiology EP,207RC0001X
5,1780847384,ASHISH,SHAH,NaN,2008-07-03,United States,MAILING,1615 N CONVENT ST STE 1,BOURBONNAIS,IL,...,MD,,DE; NC,,Gastroenterology,a3k7h000000NMXe,119.0,Gastroenterology,05-1 Gastroenterology,207RG0100X
6,1760801021,ASHISH,SHAH,NaN,2014-04-15,United States,LOCATION,1611 NW 12TH AVE,MIAMI,FL,...,MD,,,,Neurological Surgery,a3k7h000000NMXL,109.0,Neurological Surgery,03-1 Neurosurgery,207T00000X
7,1972832467,ASHISH,SHAH,NaN,2009-12-14,United States,MAILING,1313 13TH ST S,BIRMINGHAM,AL,...,MD,,,,Foot and Ankle Surgery,a3k7h000000NMX8,107.0,Orthopaedic Surgery,02-3 Ortho Subs,207XX0004X


In [73]:
data_df.to_csv("54.csv", index=False)

In [68]:
import pandas as pd

uno = pd.read_csv(
    "C:\\Users\\muhammad.madyun\\OneDrive - Hayes Locums\\Documents\\Data\\Analysis\\Sourcing\\9update.csv",
    low_memory=False,
    index_col=False,
    encoding="unicode_escape",
    dtype=object,
).fillna("")

qua = pd.read_csv(
    "C:\\Users\\muhammad.madyun\\OneDrive - Hayes Locums\\Documents\\Data\\Analysis\\Sourcing\\enrich1.csv",
    low_memory=False,
    index_col=False,
    encoding="unicode_escape",
    dtype=object,
).fillna("")

# dos = pd.read_csv(
#    "C:\\Users\\muhammad.madyun\\OneDrive - Hayes Locums\\Documents\\Data\\Analysis\\new_sp.csv",
#    low_memory=False,
#    index_col=False,
#    encoding="unicode_escape",
# ).fillna("")

quat = pd.merge_ordered(qua, uno, on="NPI", how="right").fillna("")

# tres = pd.merge_ordered(
#    quat, dos, left_on="Sub Specialty: Taxonomy Code", right_on="Tax", how="left"
# ).fillna("")

In [ ]:
quat.to_csv("113k.csv", index=False)

,NPI,Primary Specialty,Licenses (Active),Lead Record Id,LEAD RECORD ID,FIRST NAME,LAST NAME,MIDDLE NAME,ACQUISITION SOURCE,SUB SPECIALTY: TAXONOMY CODE,PRACTITIONER TYPE,SUB SPECIALTY: SPECIALTY ID,ID,AMA SPECIALTY: AMA SPECIALTY NAME,CLASSIFICATION,GL CODE,TEAM,AMA SPECIALTY: ID,TAX
0,1003009705,Family Medicine,AR; CO; NJ; NM; NV; SD; TN,00Q5b00000uBBTcEAO,,SERENA,WOODS-GRIMM,,W2-Data List,207Q00000X,MD,a1K7h000002Z2TA,a4S8G0000009pJSUAY,Family Medicine,Family Medicine,15-2 Family Practice,Hospitalist,a3k7h000000NMZX,207Q00000X
1,1003010059,Pediatric Neurology,IL,00Q5b00000u01HjEAI,,ANDREA,PARDO,CATALINA,Data Source,2084N0402X,MD,a1K7h000002Z2VW,a4S8G0000009pH7UAI,Neurology with Special Qualifications in Child...,Pediatric Neurology,09-2 Pediatric Neurology,Neurology,a3k7h000000NMYW,2084N0402X
2,1003010307,Family Medicine,CA; FL; IA; NV; OR,00Q5b00000uBBTfEAO,,JENNIFER,STONE,LYNN,Data Source,207Q00000X,MD,a1K7h000002Z2TA,a4S8G0000009pJSUAY,Family Medicine,Family Medicine,15-2 Family Practice,Hospitalist,a3k7h000000NMZX,207Q00000X
3,1003017906,Family Medicine,CA,00Q5b00000uBBU0EAO,,LINDEN,BURZELL,JOHN,Data Source,207Q00000X,MD,a1K7h000002Z2TA,a4S8G0000009pJSUAY,Family Medicine,Family Medicine,15-2 Family Practice,Hospitalist,a3k7h000000NMZX,207Q00000X
4,1003020348,,,,,JENNIFER,RIPPON,L,Data Source,207RC0200X,MD,a1K7h000002Z2TU,a4S8G0000009pJAUAY,Critical Care Medicine,Critical Care Medicine,06-1 Pulm/CCM,Pulmonary & Critical Care Medicine,a3k7h000000NMXi,207RC0200X
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113344,1992964274,Physician Assistant,TX,00Q5b00000uq3JbEAI,,DAVID,BENNETT,,W2 Data,363A00000X,MD,a1K7h000002Z2WP,a4S8G0000009pGDUAY,Physician Assistant,Physician Assistant,08-9 PA IM SUBS,APP,a3k7h000000NMYG,363A00000X
113345,1992968473,Anesthesiology,MO; NY,00Q5b00000tK7vWEAS,,MARK,VADNEY,EDWARD,Data Source,207L00000X,MD,a1K7h000002Z2Sq,a4S8G0000009pJkUAI,General Anesthesiology,Anesthesiology,19-2 Anesthesiology,Anesthesiology,a3k7h000000NMaT,207L00000X
113346,1992976534,Nurse Practitioner,NC,00Q5b00000zh6KpEAI,,CHRISTINA,KINDEL,BISHARA,Data Source,363L00000X,NP,a1K7h000002Z2W8,a4S8G0000009pGAUAY,Nurse Practitioner,Nurse Practitioner,08-6 NP IM SUBS,APP,a3k7h000000NMY0,363L00000X
113347,1992978043,Cardiology,,00Q5b00000so1FzEAI,,OLEKSANDR,SCHEVCHUCK,,Data Source,207RC0000X,MD,a1K7h000002Z2TR,a4S8G0000009pJCUAY,Cardiovascular Disease,Interventional Cardiology,10-1 Cardiology Interventional,Cardiology,a3k7h000000NMYZ,207RC0000X


In [66]:
tres[tres["Lead Record Id"] != ""]

,NPI,Primary Specialty,Licenses (Active),Lead Record Id,First Name,Last Name,Middle Name,Acquisition Source,Sub Specialty: Taxonomy Code,Practitioner Type,Sub Specialty: Specialty ID,ID,AMA SPECIALTY: AMA SPECIALTY NAME,CLASSIFICATION,GL CODE,TEAM,AMA SPECIALTY: ID,Tax,STATUS
31,1013110824,Oral & Maxillofacial Surgery,MI,00Q5b00000u9jJQEAY,LESLIE,ORZECH,ANN,W2-Data List,1223S0112X,MD,a1K7h000002Z2W5,a4S8G0000009pJrUAI,Oral and Maxillofacial Surgery,Oral & Maxillofacial Surgery,04-1 Urology,Urology/ENT/Plastic Surgery,a3k7h000000NMXP,1223S0112X,Item Created
171,1053389965,Oral & Maxillofacial Surgery,,00Q5b00000u9jJaEAI,ROBERT,MARCIANI,,W2-Data List,1223S0112X,MD,a1K7h000002Z2W5,a4S8G0000009pJrUAI,Oral and Maxillofacial Surgery,Oral & Maxillofacial Surgery,04-1 Urology,Urology/ENT/Plastic Surgery,a3k7h000000NMXP,1223S0112X,Item Created
179,1063420032,Oral & Maxillofacial Surgery,,00Q5b00000u9jJcEAI,WILLIAM,BARKER,JASON,W2-Data List,1223S0112X,MD,a1K7h000002Z2W5,a4S8G0000009pJrUAI,Oral and Maxillofacial Surgery,Oral & Maxillofacial Surgery,04-1 Urology,Urology/ENT/Plastic Surgery,a3k7h000000NMXP,1223S0112X,Item Created
254,1083687529,Plastic Surgery,CT,00Q5b00000u9osOEAQ,DEREK,STEINBACHER,MATTHEW,W2-Data List,1223S0112X,MD,a1K7h000002Z2W5,a4S8G0000009pJrUAI,Oral and Maxillofacial Surgery,Oral & Maxillofacial Surgery,04-1 Urology,Urology/ENT/Plastic Surgery,a3k7h000000NMXP,1223S0112X,Item Created
270,1093738106,Oral & Maxillofacial Surgery,KY; UT,00Q5b00000u9jMaEAI,ROBERT,FLINT,L,Data Source,1223S0112X,MD,a1K7h000002Z2W5,a4S8G0000009pJrUAI,Oral and Maxillofacial Surgery,Oral & Maxillofacial Surgery,04-1 Urology,Urology/ENT/Plastic Surgery,a3k7h000000NMXP,1223S0112X,Item Created
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
588696,1992884340,CRNA,OH,00Q5b00000uAQtoEAG,SEAN,BARCLAY,,W2 Data,367500000X,MD,a1K7h000002Z2W7,a4S8G0000009pFtUAI,CRNA,"Nurse Anesthetist, Certified Registered",08-2 CRNA,APP,a3k7h000000NMXu,367500000X,Item Created
588697,1992895460,CRNA,AR,00Q5b00000uA8s9EAC,JOHN,SAWYER,,W2 Data,367500000X,MD,a1K7h000002Z2W7,a4S8G0000009pFtUAI,CRNA,"Nurse Anesthetist, Certified Registered",08-2 CRNA,APP,a3k7h000000NMXu,367500000X,Item Created
588698,1992896864,CRNA,ND,00Q5b00000uANz5EAG,JAY,ERICKSON,,W2 Data,367500000X,MD,a1K7h000002Z2W7,a4S8G0000009pFtUAI,CRNA,"Nurse Anesthetist, Certified Registered",08-2 CRNA,APP,a3k7h000000NMXu,367500000X,Item Created
588699,1992993448,CRNA,FL; NY,00Q6g00000LtvRtEAJ,VIRGINIA,SIGNORELLI,MARIE,Data Source,367500000X,CRNA,a1K7h000002Z2W7,a4S8G0000009pFtUAI,CRNA,"Nurse Anesthetist, Certified Registered",08-2 CRNA,APP,a3k7h000000NMXu,367500000X,Item Created


In [41]:
quat[quat["Tax"] == ""]

,NPI,Lead Record Id,Tax
0,""" 15686054",00Q6g00000LOlIuEAL,
1,1003000134,00Q5b00000uqkXNEAY,
2,1003000159,00Q5b00000zhShlEAE,
3,1003000282,00Q5b00000zhHJpEAM,
4,1003000423,00Q6g00000ISEn4EAH,
...,...,...,...
1048570,2234567899,00Q5b00000sROn3EAG,
1048571,4843886809,00Q5b00000u8tTOEAY,
1048572,5678910234,00Q5b00000tLTczEAG,
1048573,9876453218,00Q6g00000ISr7LEAT,


In [40]:
tres1[tres1["Lead Record Id"] == ""]

,NPI,Lead Record Id,Tax,ID,AMA SPECIALTY: AMA SPECIALTY NAME,CLASSIFICATION,GL CODE,TEAM,AMA SPECIALTY: ID,STATUS


In [38]:
tres1.to_csv("tree.csv", index=False)

In [6]:
data_df.to_csv("144.csv", index=False)

In [15]:
import requests
import json
import csv
import pandas as pd
from pandas.io.json import json_normalize


z = pd.read_csv(
    "C:\\Users\\muhammad.madyun\\OneDrive - Hayes Locums\\Documents\\Data\\Analysis\\Sourcing\\s_name.csv",
    low_memory=False,
    index_col=False,
    encoding="unicode_escape",
).fillna("")
s = pd.read_csv(
    "C:\\Users\\muhammad.madyun\\OneDrive - Hayes Locums\\Documents\\Data\\Analysis\\sand1.csv",
    low_memory=False,
    index_col=False,
    encoding="unicode_escape",
).fillna("")
# acct = pd.read_csv(
#    "C:\\Users\\muhammad.madyun\\OneDrive - Hayes Locums\\Documents\\Data\\Analysis\\acct_npi.csv",
#    low_memory=False,
#    index_col=False,
#    encoding="unicode_escape",
# ).fillna("")
cross = pd.read_csv(
    "C:\\Users\\muhammad.madyun\\OneDrive - Hayes Locums\\Documents\\Data\\Analysis\\cross_npi.csv",
    low_memory=False,
    index_col=False,
    encoding="unicode_escape",
    dtype="object",
).fillna("")


cross = cross[cross["NPI"] != ""]
# acct1 = acct[acct["Person Account: NPI Number"] != ""]
s.rename(columns={"Specialty: Specialty Name": "Sub-Specialty"})
s = s.drop_duplicates()
s1 = s[s["Taxonomy Code"] != ""]


errors = []
data = []

# merge = pd.concat([a,b,c], ignore_index=True)
# file1 = merge[(merge['Mail Name'] != '© 2023 by the American Medical Association') & merge['Mail Name'] != 'Â© 2023 by the American Medical Association']
# file1 = file1[['Mail  Last Name', 'Mail First Name', 'MD-DO Flag', 'City', 'State', 'Zip', 'Primary Specialty']]
# file = file1.rename(
#    columns = {'Mail  Last Name': 'last_name',
#            'Mail First Name': 'first_name',
#            'Mail City': 'City',
#            'State': 'state',
#            'Zip': 'Postal Code',
#            'MD-DO Flag': 'Practitioner Type'
#            })
data_dfs = pd.DataFrame(
    columns=[
        "number",
        "First Name",
        "Last Name",
        "Gender",
        "basic.enumeration_date",
        "country_name",
        "address_purpose",
        "address_1",
        "city",
        "state_x",
        "postal_code",
        "telephone_number",
        "code",
        "taxonomy_group",
        "desc",
        "state_y",
        "license",
        "primary",
    ]
).fillna("")

data_df = pd.DataFrame(
    columns=[
        "number",
        "First Name",
        "Last Name",
        "Gender",
        "basic.enumeration_date",
        "country_name",
        "address_purpose",
        "address_1",
        "city",
        "state_x",
        "postal_code",
        "telephone_number",
        "code",
        "taxonomy_group",
        "desc",
        "state_y",
        "license",
        "primary",
    ]
).fillna("")

url = "https://npiregistry.cms.hhs.gov/api/?version=2.1"


file = z.fillna("")
file_df = file[["NPI"]]

for row in file_df.index:
    npi = file_df["NPI"][row]

    parameters = {
        "version": "2.1",
        "number": npi,
        "enumeration_type": "Organizational Providers",
        "name_purpose": "AO",
        "pretty": "true",
    }
    response = requests.get(url, params=parameters).json()
    # response.raise_for_status()
    #
    # if response.status_code != 204:
    #    return response.json()
    for i in response:
        if i == "result_count":
            if response["result_count"] < 1:
                errors.append(parameters)
            elif response["result_count"] >= 1:
                data.append(response)
        else:
            errors.append(parameters)
    for l in data:
        data1 = pd.json_normalize(
            l["results"], "addresses", ["number", "enumeration_type"]
        )
        data1 = data1.drop_duplicates(subset="number")
        data2 = pd.json_normalize(l["results"], "taxonomies", ["number"])
        data4 = pd.json_normalize(l["results"])
        file1 = pd.merge_ordered(data1, data2, on="number", how="outer")
        file2 = pd.merge_ordered(file1, data4, on="number", how="outer")

        file3 = file2[
            [
                "number",
                "basic.first_name",
                "basic.last_name",
                "basic.gender",
                "basic.enumeration_date",
                "country_name",
                "address_purpose",
                "address_1",
                "city",
                "state_x",
                "postal_code",
                "telephone_number",
                "code",
                "taxonomy_group",
                "desc",
                "state_y",
                "license",
                "primary",
            ]
        ]
        file4a = file3.rename(
            columns={
                "basic.first_name": "First Name",
                "basic.last_name": "Last Name",
                "basic.middle_name": "Middle Name",
                "basic.credential": "Practitioner Type",
                "basic.gender": "Gender",
            }
        ).drop_duplicates(subset="number")
        file4b = pd.merge_ordered(
            file4a, cross, left_on="number", right_on="NPI", how="left"
        ).fillna("")
        file4c = pd.merge_ordered(
            file4b, s1, left_on="code", right_on="Taxonomy Code", how="left"
        ).fillna("")
        file4c = file4c.replace(
            {"Practitioner Type": ""}, {"Practitioner Type": "MD"}, regex=True
        )
        file4 = file4c[file4c["Specialty: Specialty Name"] != ""].drop_duplicates(
            subset="number"
        )
        file4d = file4c[file4c["Primary Specialty"] == "Verify"]

        if len(data_df) < 50000:
            data_df = pd.concat([data_df, file4]).fillna("")
            data_dfs = pd.concat([data_dfs, file4d]).fillna("")

            data_df.to_csv("n_source.csv", index=False)

        else:
            break

In [8]:
import pandas as pd

par = pd.read_csv(
    "C:\\Users\\muhammad.madyun\\OneDrive - Hayes Locums\\Documents\\Data\\Analysis\\npi_tax.csv",
    low_memory=False,
    index_col=False,
    encoding="unicode_escape",
    dtype=object,
).fillna("")

pro = pd.read_csv(
    "C:\\Users\\muhammad.madyun\\OneDrive - Hayes Locums\\Documents\\Data\\Analysis\\ama.csv",
    low_memory=False,
    index_col=False,
    encoding="unicode_escape",
    dtype=object,
).fillna("")
import_a = pd.merge_ordered(par, pro, on="Taxonomy Code", how="left")

,CREATEDBYID,CREATEDDATE,DIRECTOR__C,GLCODE__C,ID,ISDELETED,LASTACTIVITYDATE,LASTMODIFIEDBYID,LASTMODIFIEDDATE,LASTREFERENCEDDATE,...,SPECIALTY_EXTERNAL_ID__C_x,SPECIALTY_FLOW_NAME__C,SYSTEMMODSTAMP,TAXONOMY_CODE__C,TEAM__C_x,NS_INTERNAL_ID__C_y,NAME_y,PRIMARY_SPECIALTY__C_y,SPECIALTY_EXTERNAL_ID__C_y,TEAM__C_y
0,0055b00000QlfQXAAZ,2023-07-20T17:55:02.000Z,,01-1 General Surgery,a1K7h000002Z2VuEAK,false,,0055b00000QlfQXAAZ,2023-07-20T17:55:02.000Z,,...,Spec - 202,Plastic and Reconstructive Surgery | Team: Gen...,2023-07-20T17:55:02.000Z,2086S0122X,General Surgery,101,Trauma Surgery,General Surgery,SPEC-302,General Surgery
12,0055b00000QlfQXAAZ,2023-07-20T17:55:02.000Z,,01-1 General Surgery,a1K7h000002Z2VvEAK,false,,0055b00000QlfQXAAZ,2023-07-20T17:55:02.000Z,,...,Spec - 203,Surgery of the Hand | Team: General Surgery | ...,2023-07-20T17:55:02.000Z,2086S0105X,General Surgery,101,Trauma Surgery,General Surgery,SPEC-302,General Surgery
24,0055b00000QlfQXAAZ,2023-07-20T17:55:02.000Z,,01-1 General Surgery,a1K7h000002Z2VwEAK,false,,0055b00000QlfQXAAZ,2023-07-20T17:55:02.000Z,,...,Spec - 204,Surgical Critical Care | Team: General Surgery...,2023-07-20T17:55:02.000Z,2086S0102X,General Surgery,101,Trauma Surgery,General Surgery,SPEC-302,General Surgery
36,0055b00000QlfQXAAZ,2023-07-20T17:55:02.000Z,,01-1 General Surgery,a1K7h000002Z2VxEAK,false,,0055b00000QlfQXAAZ,2023-07-20T17:55:02.000Z,,...,Spec - 205,General Surgical Oncology | Team: General Surg...,2023-07-20T17:55:02.000Z,2086X0206X,General Surgery,101,Trauma Surgery,General Surgery,SPEC-302,General Surgery
48,0055b00000QlfQXAAZ,2023-07-20T17:55:02.000Z,,01-1 General Surgery,a1K7h000002Z2W1EAK,false,,0055b00000QlfQXAAZ,2023-07-20T17:55:02.000Z,,...,Spec - 209,Transplant Surgery | Team: General Surgery | P...,2023-07-20T17:55:02.000Z,204F00000X,General Surgery,101,Trauma Surgery,General Surgery,SPEC-302,General Surgery
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1288,0055b00000QlfQXAAZ,2023-07-20T17:55:00.000Z,,19-4 Ophthalmology,a1K7h000002Z2UCEA0,false,,0055b00000QlfQXAAZ,2023-07-20T17:55:00.000Z,,...,Spec - 88,Uveitis and Ocular Inflammatory Disease | Team...,2023-07-20T17:55:00.000Z,207WX0108X,General Surgery,104,Ophthalmology,Ophthalmology,SPEC-057,General Surgery
1290,0055b00000QlfQXAAZ,2023-07-20T17:55:00.000Z,,19-5 Pediatric Endocrinology,a1K7h000002Z2UsEAK,false,,0055b00000QlfQXAAZ,2023-07-20T17:55:00.000Z,,...,Spec - 138,Pediatric Endocrinology | Team: Pediatrics | P...,2023-07-20T17:55:00.000Z,2080P0205X,Pediatrics,153,Pediatric Endocrinology,Pediatrics,SPEC-093,Pediatrics
1291,0055b00000QlfQXAAZ,2023-07-20T17:54:59.000Z,,19-6 Pediatric Anesthesiology,a1K7h000002Z2SvEAK,false,,0055b00000QlfQXAAZ,2023-07-20T17:54:59.000Z,2023-08-07T21:09:57.000Z,...,Spec - 09,Pediatric Anesthesiology | Team: Anesthesiolog...,2023-07-20T17:54:59.000Z,207LP3000X,Anesthesiology,168,Pediatric Anesthesiology,Pediatric Anesthesiology,SPEC-300,Anesthesiology
1293,0055b00000QlfQXAAZ,2023-07-20T17:55:00.000Z,,19-9 Genetics,a1K7h000002Z2UbEAK,false,,0055b00000QlfQXAAZ,2023-07-20T17:55:00.000Z,,...,Spec - 121,Molecular Genetic Pathology | Team: IM Subs | ...,2023-07-20T17:55:00.000Z,207ZP0007X,IM Subs,177,Genetics,Genetics,SPEC-022,IM Subs


In [9]:
ama.to_csv("ama.csv", index=False)

In [14]:
data_df.to_csv("55.csv", index=False)

In [ ]:
#########################1

import pandas as pd


box = pd.read_csv(
    "C:\\Users\\muhammad.madyun\\OneDrive - Hayes Locums\\Documents\\Data\\Analysis\\cross_npi.csv",
    low_memory=False,
    index_col=False,
    encoding="unicode_escape",
    dtype=object
).fillna("")

box.rename(columns={"Sub Specialty": "Specialization"})

s = pd.read_csv(
    "C:\\Users\\muhammad.madyun\\OneDrive - Hayes Locums\\Documents\\Data\\Analysis\\sand2.csv",
    usecols=[
        "NPI",
        "Provider Name Prefix Text",
        "Provider Last Name (Legal Name)",
        "Provider First Name",
        "Provider Middle Name",
        "Provider Credential Text",
        "Provider Gender Code",
        "Healthcare Provider Taxonomy Code_1",
        "Healthcare Provider Taxonomy Code_2",
        "Healthcare Provider Taxonomy Code_3",
        "Healthcare Provider Taxonomy Code_4",
        "Healthcare Provider Taxonomy Code_5",
        "Healthcare Provider Taxonomy Code_6",
        "Healthcare Provider Taxonomy Code_7",
        "Healthcare Provider Taxonomy Code_8",
        "Healthcare Provider Taxonomy Code_9",
        "Healthcare Provider Taxonomy Code_10",
        "Healthcare Provider Taxonomy Code_11",
        "Healthcare Provider Taxonomy Code_12",
        "Healthcare Provider Taxonomy Code_13",
        "Healthcare Provider Taxonomy Code_14",
        "Healthcare Provider Taxonomy Code_15",
        "Provider License Number State Code_1",
        "Provider License Number State Code_2",
        "Provider License Number State Code_3",
        "Provider License Number State Code_4",
        "Provider License Number State Code_5",
        "Provider License Number State Code_6",
        "Provider License Number State Code_7",
        "Provider License Number State Code_8",
        "Provider License Number State Code_9",
        "Provider License Number State Code_10",
        "Provider License Number State Code_11",
        "Provider License Number State Code_12",
        "Provider License Number State Code_13",
        "Provider License Number State Code_14",
        "Provider License Number State Code_15",
    ],
    #chunksize=100000,
    low_memory=False,
    index_col=False,
    encoding="unicode_escape",
    dtype=object
)  # .fillna("")
#for ch in df:
#    s = pd.concat(df).fillna("")

p1 = s[
    [
        "NPI",
        "Provider Name Prefix Text",
        "Provider Last Name (Legal Name)",
        "Provider First Name",
        "Provider Middle Name",
        "Provider Credential Text",
        "Provider Gender Code",
        "Provider License Number State Code_1",
        "Provider License Number State Code_2",
        "Provider License Number State Code_3",
        "Provider License Number State Code_4",
        "Provider License Number State Code_5",
        "Provider License Number State Code_6",
        "Provider License Number State Code_7",
        "Provider License Number State Code_8",
        "Provider License Number State Code_9",
        "Provider License Number State Code_10",
        "Provider License Number State Code_11",
        "Provider License Number State Code_12",
        "Provider License Number State Code_13",
        "Provider License Number State Code_14",
        "Provider License Number State Code_15",
    ]
].fillna("")

p1["Gender"] = p1["Provider Gender Code"].replace({"F": "Female", "M": "Male"})

p1["License (Active)"] = (
    p1[
        [
            "Provider License Number State Code_1",
            "Provider License Number State Code_2",
            "Provider License Number State Code_3",
            "Provider License Number State Code_4",
            "Provider License Number State Code_5",
            "Provider License Number State Code_6",
            "Provider License Number State Code_7",
            "Provider License Number State Code_8",
            "Provider License Number State Code_9",
            "Provider License Number State Code_10",
            "Provider License Number State Code_11",
            "Provider License Number State Code_12",
            "Provider License Number State Code_13",
            "Provider License Number State Code_14",
            "Provider License Number State Code_15",
        ]
    ]
    .astype("str")
    .agg(";".join, axis=1)
    .drop_duplicates()
    .fillna("")
)
p1["License (Active)"] = (
    p1["License (Active)"]
    .replace(
        {
            ";;;;;;;;;;;;;;$": "",
            ";;;;;;;;;;;;;$": "",
            ";;;;;;;;;;;;$": "",
            ";;;;;;;;;;;$": "",
            ";;;;;;;;;;$": "",
            ";;;;;;;;;$": "",
            ";;;;;;;;$": "",
            ";;;;;;;$": "",
            ";;;;;;$": "",
            ";;;;;$": "",
            ";;;;$": "",
            ";;;$": "",
            ";;$": "",
            ";$": "",
            "^;;;;;;;;;;;;;;": "",
            "^;;;;;;;;;;;;;": "",
            "^;;;;;;;;;;;;$": "",
            "^;;;;;;;;;;;": "",
            "^;;;;;;;;;;": "",
            "^;;;;;;;;;": "",
            "^;;;;;;;;": "",
            "^;;;;;;;": "",
            "^;;;;;;": "",
            "^;;;;;": "",
            "^;;;;": "",
            "^;;;": "",
            "^;;": "",
            "^;": "",
        },
        regex=True,
    )
    .fillna("")
)
p2 = p1[
    [
        "NPI",
        "Gender",
        "License (Active)",
        "Provider Name Prefix Text",
        "Provider Last Name (Legal Name)",
        "Provider Credential Text",
        "Provider First Name",
        "Provider Middle Name",
        "Provider Credential Text",
        "Provider Gender Code",
    ]
]


s1 = s[["NPI", "Healthcare Provider Taxonomy Code_1"]].rename(
    columns={"Healthcare Provider Taxonomy Code_1": "code"}
)
s2 = s[["NPI", "Healthcare Provider Taxonomy Code_2"]].rename(
    columns={"Healthcare Provider Taxonomy Code_2": "code"}
)
s3 = s[["NPI", "Healthcare Provider Taxonomy Code_3"]].rename(
    columns={"Healthcare Provider Taxonomy Code_3": "code"}
)
s4 = s[["NPI", "Healthcare Provider Taxonomy Code_4"]].rename(
    columns={"Healthcare Provider Taxonomy Code_4": "code"}
)
s5 = s[["NPI", "Healthcare Provider Taxonomy Code_5"]].rename(
    columns={"Healthcare Provider Taxonomy Code_5": "code"}
)
s6 = s[["NPI", "Healthcare Provider Taxonomy Code_6"]].rename(
    columns={"Healthcare Provider Taxonomy Code_6": "code"}
)
s7 = s[["NPI", "Healthcare Provider Taxonomy Code_7"]].rename(
    columns={"Healthcare Provider Taxonomy Code_7": "code"}
)
s8 = s[["NPI", "Healthcare Provider Taxonomy Code_8"]].rename(
    columns={"Healthcare Provider Taxonomy Code_8": "code"}
)
s9 = s[["NPI", "Healthcare Provider Taxonomy Code_9"]].rename(
    columns={"Healthcare Provider Taxonomy Code_9": "code"}
)
s10 = s[["NPI", "Healthcare Provider Taxonomy Code_10"]].rename(
    columns={"Healthcare Provider Taxonomy Code_10": "code"}
)
s11 = s[["NPI", "Healthcare Provider Taxonomy Code_11"]].rename(
    columns={"Healthcare Provider Taxonomy Code_11": "code"}
)
s12 = s[["NPI", "Healthcare Provider Taxonomy Code_12"]].rename(
    columns={"Healthcare Provider Taxonomy Code_12": "code"}
)
s13 = s[["NPI", "Healthcare Provider Taxonomy Code_13"]].rename(
    columns={"Healthcare Provider Taxonomy Code_13": "code"}
)
s14 = s[["NPI", "Healthcare Provider Taxonomy Code_14"]].rename(
    columns={"Healthcare Provider Taxonomy Code_14": "code"}
)
s15 = s[["NPI", "Healthcare Provider Taxonomy Code_15"]].rename(
    columns={"Healthcare Provider Taxonomy Code_15": "code"}
)

##############2

sssa = pd.concat([s1, s2, s3, s4])
sssa = sssa[sssa["code"] != ""]

sssb = pd.concat([s5, s6, s7, s8])
sssb = sssb[sssb["code"] != ""]

sssc = pd.concat([s9, s10, s11, s12, s13, s14, s15])
sssc = sssc[sssc["code"] != ""]

############1
ss1 = pd.read_csv(
    "C:\\Users\\muhammad.madyun\\OneDrive - Hayes Locums\\Documents\\Data\\Analysis\\sand1.csv",
    low_memory=False,
    index_col=False,
    encoding="unicode_escape",
    dtype=object,
).fillna("")
sss1a = pd.merge_ordered(
    sssa, ss1, left_on="code", right_on="Taxonomy Code", how="left"
).fillna("")
sss1b = pd.merge_ordered(
    sssb, ss1, left_on="code", right_on="Taxonomy Code", how="left"
).fillna("")
sss1c = pd.merge_ordered(
    sssc, ss1, left_on="code", right_on="Taxonomy Code", how="left"
).fillna("")
sss1a = sss1a[sss1a["Team"] != ""]
sss1b = sss1b[sss1b["Team"] != ""]
sss1c = sss1c[sss1c["Team"] != ""]
#
#
#
sss1d = pd.concat([sss1a, sss1b, sss1c]).drop_duplicates(subset="NPI")

sss2 = pd.merge_ordered(sss1d, p2, on="NPI", how="left").fillna("")
sss2["Practitioner Type"] = "MD"


p_cred = pd.Series(sss2["Provider Credential Text"].values.flatten())


sss2["Practitioner Type"].mask(
    p_cred.str.contains(r"DO", regex=True), "DO", inplace=True
)

sss2["Practitioner Type"].mask(
    p_spec.str.contains(r"Physician Assistant"), "PA", inplace=True
)
sss2["Practitioner Type"].mask(
    p_spec.str.contains(r"Nurse Practitioner"), "NP", inplace=True
)
sss2["Practitioner Type"].mask(p_cred.str.contains(r"MD|M.D"), "MD", inplace=True)

sss3 = pd.merge_ordered(box, sss2, left_on="NPI", right_on="NPI", how="right").fillna(
    ""
)

aq = pd.Series(sss3["Acquisition Source"].values.flatten())

sss3["Acquisition Source"].mask(
    sss3["Acquisition Source"] == "", "Data Source", inplace=True
)

sss3.to_csv("first.csv", index=False)

C:\Users\muhammad.madyun\AppData\Local\Temp\ipykernel_4636\2068207676.py:262: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sss2["Practitioner Type"].mask(
C:\Users\muhammad.madyun\AppData\Local\Temp\ipykernel_4636\2068207676.py:266: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sss2["Practitioner Type"].mask(
C:\Users\muhammad.madyun\AppData\Local\Temp\ipykernel_4636\2068207676.py:269: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sss

In [252]:
import pandas as pd

box = pd.read_csv(
    "C:\\Users\\muhammad.madyun\\OneDrive - Hayes Locums\\Documents\\Data\\Analysis\\cross_prod.csv",
    low_memory=False,
    index_col=False,
    encoding="unicode_escape",
    dtype=object,
).fillna("")
xob = pd.read_csv(
    "C:\\Users\\muhammad.madyun\\OneDrive - Hayes Locums\\Documents\\Data\\Analysis\\cross_acct.csv",
    low_memory=False,
    index_col=False,
    encoding="unicode_escape",
    dtype=object,
).fillna("")

box1 = pd.read_csv(
    "C:\\Users\\muhammad.madyun\\OneDrive - Hayes Locums\\Documents\\Data\\Analysis\\ama.csv",
    low_memory=False,
    index_col=False,
    encoding="unicode_escape",
    dtype=object,
).fillna("")

red2 = pd.read_csv(
    "C:\\Users\\muhammad.madyun\\OneDrive - Hayes Locums\\Documents\\Data\\Analysis\\Sourcing\\19k.csv",
    low_memory=False,
    index_col=False,
    encoding="unicode_escape",
    dtype=object,
).fillna("")
red2 = red2.rename(
    columns={
        "code": "Taxonomy Code",
        "number": "NPI",
        "Gender_x": "Gender",
        "Acquisition Source": "AQ",
        "Practitioner Type": "PT",
    }
)

red1 = pd.merge_ordered(box, red2, left_on="NPI", right_on="NPI", how="right").fillna(
    ""
)
der1 = pd.merge_ordered(xob, red2, left_on="NPI", right_on="NPI", how="right").fillna(
    ""
)

red = pd.merge_ordered(red1, box1, on="Taxonomy Code", how="left").fillna("")
der = pd.merge_ordered(der1, box1, on="Taxonomy Code", how="left").fillna("")
# p_cred = pd.Series(red["Provider Credential Text"].values.flatten())
p_spec = pd.Series(red["Classification"].values.flatten())
# red["Practitioner Type"].mask(
#    p_cred.str.contains(r"DO", regex=True), "DO", inplace=True
# )
red["Practitioner Type"].mask(
    p_spec.str.contains(r"Physician Assistant"), "PA", inplace=True
)
red["Practitioner Type"].mask(
    p_spec.str.contains(r"Nurse Practitioner"), "NP", inplace=True
)
red["Practitioner Type"].mask(p_spec.str.contains(r"Podiatrist"), "DPM", inplace=True)
red["Practitioner Type"].mask(
    p_spec.str.contains(r"Nurse Anesthetist"), "CRNA", inplace=True
)
red["Practitioner Type"].mask(
    ~p_spec.str.contains(r"Nurse|Assistant|Podiatrist"), "MD", inplace=True
)

# red["Gender"] = red["Gender"].replace({"F": "Female", "M": "Male"})
aq = pd.Series(box["Acquisition Source"].values.flatten())
red["Acquisition Source"].mask(
    red["Acquisition Source"] == "", "Data Source", inplace=True
)

# red1 = red[(red["Specialty: ID"] == "") | (red["Gender"] == "")]

# red.to_csv("part_prod.csv", index=False)
# red1.to_csv("typeII1.csv", index=False)
in_lead = red[red["Lead Record Id"] == ""]
up_lead = red[red["Lead Record Id"] != ""]

up_acct = der[der["Account Record Id"] != ""]

In [ ]:
red


In [251]:
in_lead.to_csv("5kin.csv", index=False)
up_lead.to_csv("14kup.csv", index=False)
up_acct.to_csv("1kacc_tup.csv", index=False)

In [253]:
up_acct

,NPI,Account Record Id,First Name,Last Name,Gender,basic.enumeration_date,country_name,address_purpose,address_1,city,...,desc,state_y,license,primary,AMA Specialty: AMA Specialty Name,AMA Specialty: ID,GL Code,Classification,Team,NS Internal Id
52,1215983978,0016g00001EAEwQAAX,TODD,HANSEN,Male,5/25/2006,United States,LOCATION,10000 W COLONIAL DR STE 495,OCOEE,...,Phlebology,FL,ME159279,TRUE,,,,,,
69,1073573671,0016g00001EALcPAAX,DAVID,REICH,Male,3/27/2006,United States,MAILING,230 N BROAD ST,PHILADELPHIA,...,Transplant Surgery,PA,MD046027L,FALSE,Transplant Surgery,a3p5b000000ilSg,01-1 General Surgery,Transplant Surgery,General Surgery,101
70,1154388809,0016g00001EAGWBAA5,JORGE,ORTIZ,Male,4/27/2006,United States,LOCATION,3000 ARLINGTON AVE,TOLEDO,...,Transplant Surgery,TX,L9407,FALSE,Transplant Surgery,a3p5b000000ilSg,01-1 General Surgery,Transplant Surgery,General Surgery,101
90,1548396419,0016g00001EAFfLAAX,SHELDON,FORMAN,Male,2/26/2007,United States,MAILING,2625 W LAKE ISLE DR,MEQUON,...,Allergy & Immunology,WI,17356,TRUE,Allergy & Immunology,a3p5b000000ilSc,18-8 Allergy and Immunology,Allergy and Immunology,IM Subs,175
96,1811965098,0016g00001kE4meAAC,PETER,DEANE,Male,3/14/2006,United States,LOCATION,300 MERIDIAN CENTRE,ROCHESTER,...,Allergy & Immunology,NY,183089,TRUE,Allergy & Immunology,a3p5b000000ilSc,18-8 Allergy and Immunology,Allergy and Immunology,IM Subs,175
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19431,1942884853,0016g00001EAHdEAAX,JORDAN,LEE,Male,5/6/2021,United States,LOCATION,11301 WILSHIRE BLVD,LOS ANGELES,...,Student in an Organized Health Care Education/...,,,TRUE,,,,,,
19450,1952808875,0015b0000224KrMAAU,FAN,YANG,Female,4/8/2018,United States,LOCATION,10400 75TH ST,KENOSHA,...,Student in an Organized Health Care Education/...,,,FALSE,,,,,,
19536,1982990388,0016g00001EAFlrAAH,MEGAN,AURORA,Female,6/23/2011,United States,LOCATION,55 FRUIT ST DEPT OF,BOSTON,...,Student in an Organized Health Care Education/...,MI,4301098494,FALSE,,,,,,
19541,1992123426,0015b00002HnogbAAB,CHARLTON,COLLIE,Male,4/2/2014,United States,LOCATION,2589 N STATE ROAD 7,LAUDERDALE LAKES,...,Student in an Organized Health Care Education/...,,,FALSE,,,,,,


In [254]:
in_lead

,NPI,Licenses (Active),Practitioner Type,Acquisition Source,Active IMLC,Lead Record Id,First Name,Last Name,Gender,basic.enumeration_date,...,desc,state_y,license,primary,AMA Specialty: AMA Specialty Name,AMA Specialty: ID,GL Code,Classification,Team,NS Internal Id
0,1043397516,,MD,Data Source,,,TIMOTHY,SCHILLER,Male,11/1/2006,...,"Dentist, Oral and Maxillofacial Pathology",OH,OH30018283,TRUE,,,,,,
1,1558408716,,MD,Data Source,,,SCOTT,GOLDSTEIN,Male,1/30/2007,...,"Dentist, Oral and Maxillofacial Pathology",NY,32711,TRUE,,,,,,
2,1578507109,,MD,Data Source,,,MICHAEL,PARSONS,Male,6/15/2006,...,"Dentist, Oral and Maxillofacial Pathology",MO,14571,TRUE,,,,,,
3,1669518932,,MD,Data Source,,,KEVIN,MCLAUGHLIN,Male,1/30/2007,...,"Dentist, Oral and Maxillofacial Pathology",CT,6173,TRUE,,,,,,
4,1770655979,,MD,Data Source,,,BRIAN,JACKSON,Male,11/14/2006,...,"Dentist, Oral and Maxillofacial Pathology",WA,6128,TRUE,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19563,1992438527,,MD,Data Source,,,ASAD,KHAWAJA,Male,7/1/2022,...,Student in an Organized Health Care Education/...,,,TRUE,,,,,,
19564,1992451447,,MD,Data Source,,,BISHOY,HANNA,Male,2/23/2022,...,Student in an Organized Health Care Education/...,,,TRUE,,,,,,
19565,1992493696,,MD,Data Source,,,FARAH,HASAN,Female,5/1/2023,...,Student in an Organized Health Care Education/...,,,TRUE,,,,,,
19567,1992962872,,MD,Data Source,,,MICEAN,JOHNIKIN,Female,5/19/2008,...,Student in an Organized Health Care Education/...,,,TRUE,,,,,,


In [255]:
up_lead

,NPI,Licenses (Active),Practitioner Type,Acquisition Source,Active IMLC,Lead Record Id,First Name,Last Name,Gender,basic.enumeration_date,...,desc,state_y,license,primary,AMA Specialty: AMA Specialty Name,AMA Specialty: ID,GL Code,Classification,Team,NS Internal Id
50,1679861611,KY; MA; RI,MD,W2-Data List,,00Q5b00000tNmFeEAK,ABDALLAH,KHARNAF,Male,7/12/2011,...,Independent Medical Examiner,RI,LP02232,FALSE,,,,,,
55,1013993054,,MD,W2-Data List,,00Q5b00000uBCkiEAG,DAVID,DUNKIN,Male,12/21/2005,...,"Neuromusculoskeletal Medicine, Sports Medicine",OH,34003493,FALSE,,,,,,
58,1023256237,MI; WA,MD,W2-Data List,,00Q5b00000u00DLEAY,BENJAMIN,ATKINSON,Male,1/26/2009,...,Neuromusculoskeletal Medicine & OMM,MI,2257995,FALSE,,,,,,
59,1205832607,OK,MD,W2-Data List,,00Q5b00000sq5bDEAQ,CHRISTIAN,ELLIS,Male,6/22/2005,...,Neuromusculoskeletal Medicine & OMM,OK,3660,FALSE,,,,,,
66,1679779532,FL; NC,MD,W2-Data List,,00Q5b00000u9jImEAI,BRENT,GOLDEN,Male,6/26/2007,...,Oral & Maxillofacial Surgery,FL,ME123445,TRUE,Oral & Maxillofacial Surgery,a3p5b000000ilTA,04-1 Urology,Oral & Maxillofacial Surgery,Urology/ENT/Plastic Surgery,114
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19556,1992327837,TX,MD,Conference - Chest 2022,No,00Q5b00000xPIGuEAO,STEPHEN,TRAN,Male,5/14/2020,...,Student in an Organized Health Care Education/...,TX,BP10071533,TRUE,,,,,,
19557,1992360986,IL,MD,W2-Data List,,00Q5b00000sqa62EAA,CORALIE,PARDO,Female,5/5/2019,...,Student in an Organized Health Care Education/...,,,FALSE,,,,,,
19558,1992364657,,MD,MMS Resident/Fellowship,,00Q5b000011FOpWEAW,JORDAN,EZELL,Male,6/6/2019,...,Student in an Organized Health Care Education/...,,,TRUE,,,,,,
19562,1992393284,AL,MD,MMS Resident/Fellowship,,00Q5b000011FbJAEA0,KAITLYN,BARR,Female,1/4/2021,...,Student in an Organized Health Care Education/...,,,TRUE,,,,,,
